<a href="https://colab.research.google.com/github/amadousysada/image-segmentation/blob/main/p8_01_exploration_and_features_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

## Import des librairies

In [1]:
import os

import numpy as np
import pandas as pd


print("Versions:")
print(f"{' '*2}- Pandas =>: {pd.__version__}")
print(f"{' '*2}- Numpy  =>: {np.__version__}")

Versions:
  - Pandas =>: 2.2.2
  - Numpy  =>: 2.0.2
